# Zipline 手續費

Zipline有三種手續費計算方法(commission models)：
    
    PerShare(cost=0.001, min_trade_cost=0.0)：按照買賣的股數算，還可以設定一個最低費用
    PerTrade(cost=0.1)：一筆交易收一個固定手續費
    PerDollar(cost=0.0015)：按照交易額抽成
    
設定方法是在def initialize(context): 裡面加入
 
    set_commission(commission.其中一種計費方法)
 
set_commission只能一次用一種計算方法，如果要混合兩種計費方法，例如每單固定收0.5，再加上每股抽成0.001，zipline建議自己在commission.py下面寫一個class，裡面要包含calculate()。

參考資料：https://zipline.ml4trading.io/api-reference.html#commission-models

## 設定環境

In [1]:
import pandas as pd
import datetime
import tejapi
import time
import os
import warnings
warnings.filterwarnings('ignore')

# tej_key-------------------------------------------
tej_key ='your key'
tejapi.ApiConfig.api_key = tej_key  
os.environ['TEJAPI_KEY'] = tej_key
os.environ['TEJAPI_BASE'] = "https://api.tej.com.tw"

# universe and benchmark----------------------------
idx=['TWN50']   # 'TM100' 'TWN50'
benchmark=['Y9997']

# date----------------------------------------------
# set date
start='2022-12-01'
end='2022-12-31'
os.environ['mdate'] = '20221201 20221231'      # start+' '+end #'20221011 20221223'

tz = 'UTC'
start_dt, end_dt = pd.Timestamp(start, tz = tz), pd.Timestamp(end, tz = tz)
# calendar------------------------------------------
calendar_name='TEJ'  # US equities  XTAI

# bundle_name---------------------------------------
bundle_name = 'tquant'

from zipline.utils.calendar_utils import get_calendar
if get_calendar(calendar_name).is_session(start_dt)==False:
    start_dt=get_calendar(calendar_name).next_open(start_dt)
    
if get_calendar(calendar_name).is_session(end_dt)==False:
    end_dt=get_calendar(calendar_name).previous_close(pd.Timestamp(end_dt))
    
from zipline.api import    *

from zipline import run_algorithm  
from zipline.finance import commission, slippage
from zipline.pipeline import Pipeline, CustomFactor
from zipline.pipeline.factors import Returns, AverageDollarVolume

from zipline.utils.run_algo import (get_transaction_detail,
                    get_data_for_alphalens,
                    get_record_vars)

from zipline.sources.TEJ_Api_Data import (get_Treasury_Return,
                                          get_Benchmark_Return)


from logbook import Logger, StderrHandler, INFO
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from zipline.utils import run_algo
from zipline.data import bundles

log_handler = StderrHandler(format_string='[{record.time:%Y-%m-%d %H:%M:%S.%f}]: ' +
                            '{record.level_name}: {record.func_name}: {record.message}',
                            level=INFO)
log_handler.push_application()
log = Logger('Algorithm')

coid = "1216 IR0001 5844"
    
os.environ['ticker'] = coid       
!zipline ingest -b tquant

Merging daily equity files:


[2023-08-15 01:14:32.107915] INFO: zipline.data.bundles.core: Ingesting tquant.


## 比較三種計算方法，注意三個initialize的差別

### class zipline.finance.commission.PerDollar(cost=0.0015)
- cost (float, optional) – The flat amount of commissions paid per dollar of equities traded. Default is a commission of 0.0015 per dollar transacted.

### class zipline.finance.commission.PerTrade(cost=0.0)
- cost (float, optional) – The flat amount of commissions paid per equity trade.

### class zipline.finance.commission.PerShare(cost=0.001, min_trade_cost=0.0)
- cost (float, optional) – The amount of commissions paid per share traded. Default is one tenth of a cent per share.

- min_trade_cost (float, optional) – The minimum amount of commissions paid per trade. Default is no minimum.

In [4]:
def initialize_perdollar(context):
    context.i = 0
    context.tickers = ['1216']
    context.asset = [symbol(ticker) for ticker in context.tickers]      
    set_slippage(slippage.FixedSlippage(spread=0.00))
    set_commission(commission.PerDollar(cost=0.001))
    set_benchmark(symbol('IR0001'))
    
def initialize_pertrade(context):
    context.i = 0
    context.tickers = ['1216']
    context.asset = [symbol(ticker) for ticker in context.tickers]      
    set_slippage(slippage.FixedSlippage(spread=0.00))
    set_commission(commission.PerTrade(cost=0.5))
    set_benchmark(symbol('IR0001'))
    
def initialize_pershare(context):
    context.i = 0
    context.tickers = ['1216']
    context.asset = [symbol(ticker) for ticker in context.tickers]      
    set_slippage(slippage.FixedSlippage(spread=0.00))
    set_commission(commission.PerShare(cost=0.001, min_trade_cost=5.0))
    set_benchmark(symbol('IR0001'))
    
    
def handle_data(context, data):

    if context.i == 0:
        for asset in context.asset:
            order_target(asset, 1000)
            
    record(close=data.current(context.asset, 'close'))
    context.i += 1
    
def analyze(context, perf):

    pass

capital_base = 1e5
treasury_returns = get_Treasury_Return(start = start_dt,
                                      end = end_dt,
                                      rate_type = 'Time_Deposit_Rate',                     
                                      term = '1y',
                                      symbol = '5844')


In [5]:
closing_price = tejapi.get('TWN/APIPRCD',coid=['1216'], opts={'columns':['mdate','coid','close_d']}, mdate={'gte':start_dt,'lte':end_dt }, paginate=True)

perdollar = run_algorithm(start=start_dt,
                            end=end_dt,
                            initialize=initialize_perdollar,
                            handle_data=handle_data,
                            capital_base=capital_base,
                            analyze=analyze,
                            treasury_returns=treasury_returns,
                            trading_calendar=get_calendar(calendar_name),
                            bundle=bundle_name)

pertrade = run_algorithm(start=start_dt,
                            end=end_dt,
                            initialize=initialize_pertrade,
                            handle_data=handle_data,
                            capital_base=capital_base,
                            analyze=analyze,
                            treasury_returns=treasury_returns,
                            trading_calendar=get_calendar(calendar_name),
                            bundle=bundle_name)

pershare = run_algorithm(start=start_dt,
                            end=end_dt,
                            initialize=initialize_pershare,
                            handle_data=handle_data,
                            capital_base=capital_base,
                            analyze=analyze,
                            treasury_returns=treasury_returns,
                            trading_calendar=get_calendar(calendar_name),
                            bundle=bundle_name)

[2023-08-15 01:15:03.074603]: INFO: handle_simulation_end: Simulated 22 trading days
first open: 2022-12-01 01:01:00+00:00
last close: 2022-12-30 05:30:00+00:00
[2023-08-15 01:15:03.173243]: INFO: handle_simulation_end: Simulated 22 trading days
first open: 2022-12-01 01:01:00+00:00
last close: 2022-12-30 05:30:00+00:00
[2023-08-15 01:15:03.272552]: INFO: handle_simulation_end: Simulated 22 trading days
first open: 2022-12-01 01:01:00+00:00
last close: 2022-12-30 05:30:00+00:00


同樣在12/1下單一張統一 (1216) 股票，如果用PerDollar算法，就是第二天12/2收盤價65 * 1000 * 0.001 = 65，如果是PerOrder，就固定是預先設定好的0.5。PerShare雖然設定一股抽0.001，但是1000 * 0.001 = 1 小於最低費用 min_trade_cost = 5，所以手續費是5。

註：價格以成交日收盤價為準，數量以成交時為準，也就是說，如果因為split造成amount有任何變化，計算上都是用成交時新的amount。

In [6]:
perdollar['orders'][1]

[{'id': 'd45e2be1af584f919ac64518072072d8',
  'dt': Timestamp('2022-12-02 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2022-12-01 13:30:00+0800', tz='Asia/Taipei'),
  'amount': 1000,
  'filled': 1000,
  'commission': 65.0,
  'stop': None,
  'limit': None,
  'stop_reached': False,
  'limit_reached': False,
  'sid': Equity(0 [1216]),
  'status': <ORDER_STATUS.FILLED: 1>}]

In [7]:
pertrade['orders'][1]

[{'id': 'a4f1ebe02f4048d2ade0fe58001a37d0',
  'dt': Timestamp('2022-12-02 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2022-12-01 13:30:00+0800', tz='Asia/Taipei'),
  'amount': 1000,
  'filled': 1000,
  'commission': 0.5,
  'stop': None,
  'limit': None,
  'stop_reached': False,
  'limit_reached': False,
  'sid': Equity(0 [1216]),
  'status': <ORDER_STATUS.FILLED: 1>}]

In [8]:
pershare['orders'][1]

[{'id': 'ddd1619648f544afaa474ae5536b92bb',
  'dt': Timestamp('2022-12-02 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2022-12-01 13:30:00+0800', tz='Asia/Taipei'),
  'amount': 1000,
  'filled': 1000,
  'commission': 5.0,
  'stop': None,
  'limit': None,
  'stop_reached': False,
  'limit_reached': False,
  'sid': Equity(0 [1216]),
  'status': <ORDER_STATUS.FILLED: 1>}]

In [9]:
closing_price[0:2]

,mdate,coid,close_d
None,,,
0,2022-12-01,1216,65.9
1,2022-12-02,1216,65.0
